# 1) 

# Thi

In [2]:
def fib():
    a, b = 1, 1
    while True:
        a, b = b, a + b
        yield b

f = fib()
n = 2
while len(str(next(f))) < 1000:
    n += 1
n += 1 # +1 because we don't count the last iteration
print(n)

4782


In [7]:
def run_of_primes(a, b):
    n = 0
    while n**2 + a * n + b in Primes():
        n += 1
    return n - 1

In [8]:
longest_run = 0
for a in range(-1000, 1000):
    for b in range(-1000, 1000):
        if run_of_primes(a, b) >= longest_run:
            longest_run = run_of_primes(a, b)
            longest_a = a
            longest_b = b
print(f'{longest_run=} with: {longest_a=}, {longest_b=} and product {longest_a * longest_b}')

longest_run=70 with: longest_a=-61, longest_b=971 and product -59231


## 23) Non-abundant sums


A perfect number is a number for which the sum of its proper divisors is exactly equal to the number. For example, the sum of the proper divisors of 28 would be 1 + 2 + 4 + 7 + 14 = 28, which means that 28 is a perfect number.

A number n is called deficient if the sum of its proper divisors is less than n and it is called abundant if this sum exceeds n.

As 12 is the smallest abundant number, 1 + 2 + 3 + 4 + 6 = 16, the smallest number that can be written as the sum of two abundant numbers is 24. By mathematical analysis, it can be shown that all integers greater than 28123 can be written as the sum of two abundant numbers. However, this upper limit cannot be reduced any further by analysis even though it is known that the greatest number that cannot be expressed as the sum of two abundant numbers is less than this limit.

Find the sum of all the positive integers which cannot be written as the sum of two abundant numbers.


In [18]:
def p23():
    upper_bound = 28123
    answer = 0
    # only on a subset of abundant numbers, {a}, up to the number, n, itself (to cut down on execution time)
    # check if n - a is not in the set of abundant numbers, {a}, for all a in {a}
    abundant_numbers = set()
    for n in range(1,upper_bound + 1):
        if n < sum(divisors(n)[:-1]): # the sum of properdivisors of n
            abundant_numbers.add(n)
        if not any((n - a in abundant_numbers) for a in abundant_numbers):
            answer += n
    return answer
print(p23())
#%timeit p23()

4179871


In [10]:
# Other implementations from thread:
def p23_alternative1():
    LIMIT = 28124
    divisorsum = [0] * LIMIT
    for i in range(1, LIMIT):
        for j in range(i * 2, LIMIT, i):
            divisorsum[j] += i
    abundantnums = [i for (i, x) in enumerate(divisorsum) if x > i]

    expressible = [False] * LIMIT
    for i in abundantnums:
        for j in abundantnums:
            if i + j < LIMIT:
                expressible[i + j] = True
            else:
                break

    print(sum(i for (i, x) in enumerate(expressible) if not x))

4179871


## 24) Lexicographic Permutations

A permutation is an ordered arrangement of objects. For example, 3124 is one possible permutation of the digits 1, 2, 3 and 4. If all of the permutations are listed numerically or alphabetically, we call it lexicographic order. The lexicographic permutations of 0, 1 and 2 are:

012   021   102   120   201   210

What is the millionth lexicographic permutation of the digits 0, 1, 2, 3, 4, 5, 6, 7, 8 and 9?


In [81]:
import itertools
perms = itertools.permutations('0123456789')
for i in range(1_000_000-1):
    next(perms)
print(next(perms))

# OR #

def get_nth_permutation(init_list, n):
    """
    This function uses the fact that the m'th element in a list will be on the first space 
    in the permutation, when (m-1)*(len(list)-1)! <= perm_number < m*(len(list)-1)!. 
    Hence we can iteratively find the elements in the permuted list by poping them from the 
    initial list, when we find they obey above inequality, and then carry over the remainder,
    and use that remainder as the perm_number of the new shorter list...
    """
    perm_list = list()
    n = n % factorial(len(init_list)) - 1 # Make sure we do not loop around so divide n by total number of permutations
    while len(init_list):
        d = len(init_list) - 1
        perm_list.append(init_list.pop(n // factorial(d)) )
        n = n % factorial(d)
    return perm_list

print(get_nth_permutation(list('0123456789'), 1_000_000))

('2', '7', '8', '3', '9', '1', '5', '4', '6', '0')
['2', '7', '8', '3', '9', '1', '5', '4', '6', '0']


## 26) Reciprocal cycles



A unit fraction contains 1 in the numerator. The decimal representation of the unit fractions with denominators 2 to 10 are given:

    1/2	= 	0.5
    1/3	= 	0.(3)
    1/4	= 	0.25
    1/5	= 	0.2
    1/6	= 	0.1(6)
    1/7	= 	0.(142857)
    1/8	= 	0.125
    1/9	= 	0.(1)
    1/10	= 	0.1 

Where 0.1(6) means 0.166666..., and has a 1-digit recurring cycle. It can be seen that 1/7 has a 6-digit recurring cycle.

Find the value of d < 1000 for which 1/d contains the longest recurring cycle in its decimal fraction part.


In [35]:
def unit_fraction_string(d, max_digits):
    carry = 10
    string = '0.'
    while len(string) < max_digits + 2:
        string += str(carry//d)
        carry = carry%d * 10
        if carry == 0:
            break
    return string
unit_fraction_string(9,10)

'0.1111111111'

In [36]:
def divide(a, b):
  '''Returns the decimal representation of the fraction a / b in three parts:
  integer part, non-recurring fractional part, and recurring part.'''
  assert b > 0
  integer = a // b
  remainder = a % b
  seen = {remainder: 0}  # Holds position where each remainder was first seen.
  digits = []
  while(True):  # Loop executed at most b times (as remainders must be distinct)
    remainder *= 10
    digits.append(remainder // b)
    remainder = remainder % b
    if remainder in seen:  # Digits have begun to recur.
      where = seen[remainder]
      return (integer, digits[:where], digits[where:])
    else:
      seen[remainder] = len(digits)

In [44]:
max_length = 0
max_d = 1
for d in range(1, 1000):
    integer, non_recurring, recurring = divide(1,d)
    if len(recurring) > max_length:
        max_length = len(recurring)
        max_d = d
print(max_d)

983


## 28) Number spiral diagonals

Starting with the number 1 and moving to the right in a clockwise direction a 5 by 5 spiral is formed as follows:

```verbatim
21 22 23 24 25
20  7  8  9 10
19  6  1  2 11
18  5  4  3 12
17 16 15 14 13
```

It can be verified that the sum of the numbers on the diagonals is 101.

What is the sum of the numbers on the diagonals in a 1001 by 1001 spiral formed in the same way?


In [89]:
def sum_of_diagonals(width):
    assert width > 0
    assert width % 2 == 1
    result = 1
    n = 1
    i = 2
    while i < width:
        for _ in range(4):
            n += i
            result += n
        i = i + 2
    return result

print(sum_of_diagonals(1001))

669171001


## 31) Coin Sums



In the United Kingdom the currency is made up of pound (£) and pence (p). There are eight coins in general circulation:

    1p, 2p, 5p, 10p, 20p, 50p, £1 (100p), and £2 (200p).

It is possible to make £2 in the following way:

    1×£1 + 1×50p + 2×20p + 1×5p + 1×2p + 3×1p

How many different ways can £2 be made using any number of coins?


In [111]:
x = var('x')
gen_func = 1/((1 - x)*(1 - x^2)*(1 - x^5)*(1 - x^10)*(1 - x^20)*(1 - x^50)*(1 - x^100)*(1 - x^200))
print(((1 - x)*(1 - x^2)*(1 - x^5)*(1 - x^10)*(1 - x^20)*(1 - x^50)*(1 - x^100)*(1 - x^200)).expand())

x^388 - x^387 - x^386 + x^385 - x^383 + x^382 + x^381 - x^380 - x^378 + x^377 + x^376 - x^375 + x^373 - x^372 - x^371 + x^370 - x^368 + x^367 + x^366 - x^365 + x^363 - x^362 - x^361 + x^360 + x^358 - x^357 - x^356 + x^355 - x^353 + x^352 + x^351 - x^350 - x^338 + x^337 + x^336 - x^335 + x^333 - x^332 - x^331 + x^330 + x^328 - x^327 - x^326 + x^325 - x^323 + x^322 + x^321 - x^320 + x^318 - x^317 - x^316 + x^315 - x^313 + x^312 + x^311 - x^310 - x^308 + x^307 + x^306 - x^305 + x^303 - x^302 - x^301 + x^300 - x^288 + x^287 + x^286 - x^285 + x^283 - x^282 - x^281 + x^280 + x^278 - x^277 - x^276 + x^275 - x^273 + x^272 + x^271 - x^270 + x^268 - x^267 - x^266 + x^265 - x^263 + x^262 + x^261 - x^260 - x^258 + x^257 + x^256 - x^255 + x^253 - x^252 - x^251 + x^250 + x^238 - x^237 - x^236 + x^235 - x^233 + x^232 + x^231 - x^230 - x^228 + x^227 + x^226 - x^225 + x^223 - x^222 - x^221 + x^220 - x^218 + x^217 + x^216 - x^215 + x^213 - x^212 - x^211 + x^210 + x^208 - x^207 - x^206 + x^205 - x^203 + 

## 32) Pandigital Products

We shall say that an n-digit number is pandigital if it makes use of all the digits 1 to n exactly once; for example, the 5-digit number, 15234, is 1 through 5 pandigital.

The product 7254 is unusual, as the identity, 39 × 186 = 7254, containing multiplicand, multiplier, and product is 1 through 9 pandigital.

Find the sum of all products whose multiplicand/multiplier/product identity can be written as a 1 through 9 pandigital.
HINT: Some products can be obtained in more than one way so be sure to only include it once in your sum.


In [25]:
from itertools import permutations
import numpy as np

solutions = []

numbers = [str(i) for i in range(1,10)]

for len_a in range(1, 3):
    for len_b in range(len_a, 5):
        for permutation in permutations(numbers):
            a = int(''.join(permutation[0:len_a]))
            b = int(''.join(permutation[len_a:len_b+len_a]))
            c = int(''.join(permutation[len_b+len_a:]))
            if a * b == c:
                solutions.append({'a': a, 'b': b, 'c': c})

print('Result:', sum(set(solution['c'] for solution in solutions)))

Result: 45228


## 34) Digit Factorials

145 is a curious number, as 1! + 4! + 5! = 1 + 24 + 120 = 145.

Find the sum of all numbers which are equal to the sum of the factorial of their digits.

Note: As 1! = 1 and 2! = 2 are not sums they are not included.

PS: 9! = 362880, so max number of digits is 7, since if more than this, the digit factorial sum will always be less than the number itself as 7 * 9! = 2540160. This number is in fact the limit wee need to check to. 

In [9]:
solutions = []
for i in range(7 * factorial(9)):
    digits_fact_sum = sum(factorial(int(digit)) for digit in str(i))
    if i == digits_fact_sum:
        print(i)
        solutions.append(i)
print(solutions)

1
2
145
40585
[1, 2, 145, 40585]


In [14]:
7 * factorial(9)

2540160